In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:10])

gsk_oZMXNP


#### 1. CommaSeparatedListOutputParser

In [2]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

c:\Users\KOSTA\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-tQs1zK9u-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [14]:

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# OpenAI 모델 설정
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    #model="openai/gpt-oss-120b",
    temperature=0.7
)
print(llm.model_name)


moonshotai/kimi-k2-instruct-0905


In [4]:

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | llm | output_parser

# "AI 관련 기술"에 대한 체인 호출 실행
result = chain.invoke({"subject": "AI 관련 기술"})

# 쉼표로 구분된 리스트 출력
print("AI 관련 기술 목록:")
print(result)

AI 관련 기술 목록:
['Machine Learning', 'Deep Learning', 'Natural Language Processing', 'Computer Vision', 'Reinforcement Learning']


In [5]:

# 결과 활용 예시: CSV 파일로 저장
csv_filename = "../data/ai_technologies.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["AI 기술"])  # 헤더 추가
    for item in result:
        writer.writerow([item])

print(f" '{csv_filename}' 파일로 저장 완료!")


 '../data/ai_technologies.csv' 파일로 저장 완료!


#### 2. JsonOutputParser

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# OpenAI 모델 설정
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})
print(type(response))

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))


#### 3. PandasDataFrameOutputParser

In [15]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import re

# Titanic 데이터셋 로드
df = pd.read_csv('../data/titanic.csv')

# ChatOpenAI 모델 초기화
#llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)

# 형식 지침 출력
format_instructions = parser.get_format_instructions()
print("Format Instructions:\n", format_instructions)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)

# 체인 생성
chain = prompt | llm | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

    # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

    #Please tell me the average value of the Fare column.
    print('Fare 컬럼의 평균값 출력')
    df_query3 = "Show me the average value of the Fare column."

    parser_output3 = chain.invoke({"query": df_query3})
    print(parser_output3)

except Exception as e:
    print(f"오류 발생: {e}")


Format Instructions:
 The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2,

In [ ]:

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)

# 체인 생성
chain = prompt | llm | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

    # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

    #Please tell me the average value of the Fare column.
    print('Fare 컬럼의 평균값 출력')
    df_query3 = "Show me the average value of the Fare column."

    parser_output3 = chain.invoke({"query": df_query3})
    print(parser_output3)

except Exception as e:
    print(f"오류 발생: {e}")


In [31]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# OpenAI 모델 초기화
#llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    #model="openai/gpt-oss-120b",
    temperature=0
)

# 응답 스키마 정의 data = [{},{},{}]
response_schemas = [
    #ResponseSchema(name="data", description="A list of dictionaries representing table rows."),
    ResponseSchema(name="data", description="테이블 행을 나타내는 딕셔너리들의 리스트"),
]

# Output Parser 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    당신은 테이블 형태의 데이터를 생성하는 AI 어시스턴트입니다(You are an AI assistant that generates tabular data).
    반드시 다음 스키마를 따르는 JSON 형식으로 데이터를 반환해야 합니다(You must return the data in JSON format that follows this schema):

    {format_instructions}
        
    **User Query:**
    {query}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 생성 (프롬프트 → 모델 → StructuredOutputParser)
chain = prompt | llm | parser

# 실행 함수
def generate_dataframe(user_query):
    try:
        # 모델 호출
        json_response = chain.invoke({"query": user_query})
        print(json_response)

        # 모델이 반환한 JSON을 Pandas DataFrame으로 변환
        df = pd.DataFrame(json_response["data"])

        # 결과 출력
        print("\n Generated DataFrame:\n")
        return df

    except Exception as e:
        print(f" 오류 발생: {e}")
        return None

In [19]:
# [예제 1] 2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
print('2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성')
df_seoul_housing = generate_dataframe(
    #"Create a dataset of the average apartment sale prices in Seoul for the first half of 2024 with columns: District (구), Average Price (in KRW), Number of Transactions, and Year-over-Year Change (%)."
    "2024년 상반기 서울 아파트 평균 매매가격 데이터셋을 생성해주세요.\
          컬럼은 다음과 같습니다: 구(District), 평균가격_억원(Average_Price_100M_KRW), 거래건수(Transactions), 전년동기대비증감률_퍼센트(YoY_Change_Percent).\
              서울의 25개 구에 대한 현실적인 데이터를 생성해주세요."
)
df_seoul_housing


2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
{'data': [{'District': '강남구', 'Average_Price_100M_KRW': '38.5', 'Transactions': '1,650', 'YoY_Change_Percent': '4.8'}, {'District': '서초구', 'Average_Price_100M_KRW': '36.2', 'Transactions': '1,420', 'YoY_Change_Percent': '5.3'}, {'District': '송파구', 'Average_Price_100M_KRW': '28.7', 'Transactions': '1,880', 'YoY_Change_Percent': '3.9'}, {'District': '강동구', 'Average_Price_100M_KRW': '24.3', 'Transactions': '1,560', 'YoY_Change_Percent': '4.2'}, {'District': '용산구', 'Average_Price_100M_KRW': '33.1', 'Transactions': '920', 'YoY_Change_Percent': '2.7'}, {'District': '마포구', 'Average_Price_100M_KRW': '22.9', 'Transactions': '1,240', 'YoY_Change_Percent': '3.5'}, {'District': '성동구', 'Average_Price_100M_KRW': '20.4', 'Transactions': '1,080', 'YoY_Change_Percent': '3.1'}, {'District': '동작구', 'Average_Price_100M_KRW': '18.7', 'Transactions': '1,320', 'YoY_Change_Percent': '2.9'}, {'District': '광진구', 'Average_Price_100M_KRW': '17.8', 'Transactions': '1,160', 'YoY_Cha

,District,Average_Price_100M_KRW,Transactions,YoY_Change_Percent
0,강남구,38.5,"1,650",4.8
1,서초구,36.2,"1,420",5.3
2,송파구,28.7,"1,880",3.9
3,강동구,24.3,"1,560",4.2
4,용산구,33.1,920,2.7
5,마포구,22.9,"1,240",3.5
6,성동구,20.4,"1,080",3.1
7,동작구,18.7,"1,320",2.9
8,광진구,17.8,"1,160",3.3
9,강북구,14.2,980,2.4


In [ ]:
df_seoul_housing

In [20]:
print('2024년 서울 지하철역별 유동 인구 데이터')
# [예제 2] 2024년 서울 지하철역별 유동 인구 데이터
df_seoul_subway = generate_dataframe(
    #"Generate a dataset of the top 10 busiest subway stations in Seoul in 2024 with columns: Station Name, Line Number, Daily Passenger Volume, and Weekday vs Weekend Ratio."
    "2024년 서울 지하철 승객 이용량이 가장 많은 상위 10개 역의 데이터셋을 생성해주세요. \
        컬럼은 다음과 같습니다: 역명(Station_Name), 호선(Line_Number),\
              일평균승객수_만명(Daily_Passengers_10K), 평일대주말비율(Weekday_Weekend_Ratio).\
                  실제 서울 지하철의 주요 역들을 기반으로 현실적인 데이터를 생성해주세요."
)

#if df_seoul_subway is not None:
df_seoul_subway.head()


2024년 서울 지하철역별 유동 인구 데이터
{'data': [{'Station_Name': '강남', 'Line_Number': 2, 'Daily_Passengers_10K': 21.3, 'Weekday_Weekend_Ratio': 1.18}, {'Station_Name': '신도림', 'Line_Number': 2, 'Daily_Passengers_10K': 19.7, 'Weekday_Weekend_Ratio': 1.22}, {'Station_Name': '서울역', 'Line_Number': 1, 'Daily_Passengers_10K': 18.9, 'Weekday_Weekend_Ratio': 1.3}, {'Station_Name': '왕십리', 'Line_Number': 2, 'Daily_Passengers_10K': 17.8, 'Weekday_Weekend_Ratio': 1.15}, {'Station_Name': '신림', 'Line_Number': 2, 'Daily_Passengers_10K': 17.1, 'Weekday_Weekend_Ratio': 1.1}, {'Station_Name': '홍대입구', 'Line_Number': 2, 'Daily_Passengers_10K': 16.6, 'Weekday_Weekend_Ratio': 0.95}, {'Station_Name': '교대', 'Line_Number': 2, 'Daily_Passengers_10K': 16.2, 'Weekday_Weekend_Ratio': 1.13}, {'Station_Name': '잠실', 'Line_Number': 2, 'Daily_Passengers_10K': 15.9, 'Weekday_Weekend_Ratio': 1.05}, {'Station_Name': '건대입구', 'Line_Number': 2, 'Daily_Passengers_10K': 15.4, 'Weekday_Weekend_Ratio': 1.02}, {'Station_Name': '종로3가', 'Line_Nu

,Station_Name,Line_Number,Daily_Passengers_10K,Weekday_Weekend_Ratio
0,강남,2,21.3,1.18
1,신도림,2,19.7,1.22
2,서울역,1,18.9,1.30
3,왕십리,2,17.8,1.15
4,신림,2,17.1,1.10


In [21]:
df_seoul_subway

,Station_Name,Line_Number,Daily_Passengers_10K,Weekday_Weekend_Ratio
0,강남,2,21.3,1.18
1,신도림,2,19.7,1.22
2,서울역,1,18.9,1.30
3,왕십리,2,17.8,1.15
4,신림,2,17.1,1.10
5,홍대입구,2,16.6,0.95
6,교대,2,16.2,1.13
7,잠실,2,15.9,1.05
8,건대입구,2,15.4,1.02
9,종로3가,1,15.0,1.25


In [34]:
print('한국 5대 편의점 브랜드별 2023년 매출 및 점포 수')
# [예제 3] 한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
df_korean_convenience_stores = generate_dataframe(
    #"Create a dataset of the top 5 convenience store brands in Korea in 2024 with columns: Brand Name, Number of Stores, Total Revenue (in billion KRW), and Market Share (%)."
    "2023년 한국의 편의점 브랜드 상위 5개사 데이터셋을 생성해주세요.\
          컬럼은 다음과 같습니다: 브랜드명(Brand_Name), 점포수(Store_Count), \
            총매출_조원(Revenue_Trillion_KRW), 시장점유율_퍼센트(Market_Share_Percent).\
                  CU, GS25, 세븐일레븐, 이마트24, 미니스톱 등 실제 한국 편의점 브랜드를 기반으로 현실적인 데이터를 생성해주세요."
)
df_korean_convenience_stores

한국 5대 편의점 브랜드별 2023년 매출 및 점포 수
{'data': [{'Brand_Name': 'CU', 'Store_Count': '15000', 'Revenue_Trillion_KRW': '7.2', 'Market_Share_Percent': '35.1'}, {'Brand_Name': 'GS25', 'Store_Count': '16500', 'Revenue_Trillion_KRW': '6.8', 'Market_Share_Percent': '33.2'}, {'Brand_Name': '세븐일레븐', 'Store_Count': '11000', 'Revenue_Trillion_KRW': '4.1', 'Market_Share_Percent': '20.0'}, {'Brand_Name': '이마트24', 'Store_Count': '5200', 'Revenue_Trillion_KRW': '1.8', 'Market_Share_Percent': '8.8'}, {'Brand_Name': '미니스톱', 'Store_Count': '2500', 'Revenue_Trillion_KRW': '0.6', 'Market_Share_Percent': '2.9'}]}

 Generated DataFrame:



,Brand_Name,Store_Count,Revenue_Trillion_KRW,Market_Share_Percent
0,CU,15000,7.2,35.1
1,GS25,16500,6.8,33.2
2,세븐일레븐,11000,4.1,20.0
3,이마트24,5200,1.8,8.8
4,미니스톱,2500,0.6,2.9


In [36]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.exceptions import OutputParserException # 오류 처리 개선을 위해 추가

# --------------------------------------------------------
# 1. LLM 초기화 (사용자 설정 유지)
# --------------------------------------------------------
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트 예시
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

print(f"사용 모델: {llm.model_name}")

# --------------------------------------------------------
# 2. 응답 스키마 정의
# --------------------------------------------------------
# 최상위 'data' 리스트만 정의하여 LLM이 내부 딕셔너리의 키를 자유롭게 생성하도록 합니다.
response_schemas = [
    ResponseSchema(name="data", description="테이블 행을 나타내는 딕셔너리들의 리스트"),
]

# Output Parser 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# --------------------------------------------------------
# 3. 프롬프트 템플릿 수정 (참고 자료 명시)
# --------------------------------------------------------
prompt = PromptTemplate(
    template="""
    당신은 테이블 형태의 데이터를 생성하는 AI 어시스턴트입니다.
    **반드시 아래 [참고 자료]를 기반으로 정보를 추출하여** 다음 스키마를 따르는 JSON 형식으로 데이터를 반환해야 합니다:

    {format_instructions}
        
    **[참고 자료 - 2024년 한국 5대 편의점 추정 데이터]**
    CU: 점포수 18,100개, 총매출 8.8조 원, 시장점유율 33.5%
    GS25: 점포수 17,800개, 총매출 8.5조 원, 시장점유율 32.5%
    세븐일레븐: 점포수 13,800개, 총매출 5.1조 원, 시장점유율 19.5%
    이마트24: 점포수 7,000개, 총매출 2.4조 원, 시장점유율 9.0%
    미니스톱: 점포수 2,500개, 총매출 1.0조 원, 시장점유율 3.5%
    
    **User Query:**
    {query}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# --------------------------------------------------------
# 4. 체인 생성 및 실행 함수
# --------------------------------------------------------
# 체인 생성 (프롬프트 → 모델 → StructuredOutputParser)
chain = prompt | llm | parser

# 실행 함수
def generate_dataframe(user_query):
    try:
        # 모델 호출
        json_response = chain.invoke({"query": user_query})

        # 모델이 반환한 JSON을 Pandas DataFrame으로 변환
        # json_response가 딕셔너리임을 가정하고 'data' 키에서 리스트를 추출
        df = pd.DataFrame(json_response["data"])

        # 결과 출력
        print("\n Generated DataFrame:\n")
        return df

    except OutputParserException as e:
        print(f" [오류 발생 - JSON 파싱 실패]: 모델이 유효한 JSON 스키마를 따르지 않았습니다. 오류: {e}")
        return None
    except Exception as e:
        print(f" [일반 오류 발생]: {e}")
        return None

# --------------------------------------------------------
# 5. 실행 예제 (수정된 쿼리 포함)
# --------------------------------------------------------

print('\n--- 한국 5대 편의점 데이터 생성 시작 ---')

# 쿼리: 참고 자료를 기반으로 데이터를 추출하도록 명시
query_text = (
    "위 참고 자료를 기반으로 데이터셋을 생성해주세요. "
    "컬럼은 다음과 같습니다: 브랜드명(Brand_Name), 점포수(Store_Count) - 숫자만, 총매출_조원(Revenue_Trillion_KRW) - 소수점, 시장점유율_퍼센트(Market_Share_Percent) - 숫자만."
)

df_korean_convenience_stores = generate_dataframe(query_text)

if df_korean_convenience_stores is not None:
    print(df_korean_convenience_stores)

사용 모델: moonshotai/kimi-k2-instruct-0905

--- 한국 5대 편의점 데이터 생성 시작 ---

 Generated DataFrame:

  Brand_Name  Store_Count  Revenue_Trillion_KRW  Market_Share_Percent
0         CU        18100                   8.8                  33.5
1       GS25        17800                   8.5                  32.5
2      세븐일레븐        13800                   5.1                  19.5
3      이마트24         7000                   2.4                   9.0
4       미니스톱         2500                   1.0                   3.5
